In [2]:
import sympy as sym
import numpy as np
import scipy as sp
import mpmath as mp
init_printing(use_unicode=True)

NameError: name 'init_printing' is not defined

In [51]:
kx, ky, kz = sym.symbols('kx ky kz')
sig = 0.5
theta = 90

kc = 1000
p = 2
r = 1/(1 + (sym.sqrt(kx**2 + ky**2)/kc)**p) #ADD BACK IN kz?
r=1

H = sym.Matrix([[0, -1j*sig, 0, -1j*r, 0, 0, 0, 0, 0],
           [1j*sig, 0, 0, 0, -1j*r, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, -1j*r, 0, 0, 0],
           [1j*r, 0, 0, 0, 0, 0, 0, kz, -ky],
           [0, 1j*r, 0, 0, 0, 0, -kz, 0, kx],
           [0, 0, 1j*r, 0, 0, 0, ky, -kx, 0],
           [0, 0, 0, 0, -kz, ky, 0, 0, 0],
           [0, 0, 0, kz, 0, -kx, 0, 0, 0],
           [0, 0, 0, -ky, kx, 0, 0, 0, 0]])

# k1 = kx
# k2 = sin(theta)*ky + cos(theta)*kz

dHdkx = sym.diff(H, kx)
dHdky = sym.diff(H, ky)
dHdkz = sym.diff(H, kz)

In [52]:
H

⎡  0    -0.5⋅ⅈ    0    -1.0⋅ⅈ    0       0      0    0    0 ⎤
⎢                                                           ⎥
⎢0.5⋅ⅈ    0       0      0     -1.0⋅ⅈ    0      0    0    0 ⎥
⎢                                                           ⎥
⎢  0      0       0      0       0     -1.0⋅ⅈ   0    0    0 ⎥
⎢                                                           ⎥
⎢1.0⋅ⅈ    0       0      0       0       0      0   kz   -ky⎥
⎢                                                           ⎥
⎢  0    1.0⋅ⅈ     0      0       0       0     -kz   0   kx ⎥
⎢                                                           ⎥
⎢  0      0     1.0⋅ⅈ    0       0       0     ky   -kx   0 ⎥
⎢                                                           ⎥
⎢  0      0       0      0      -kz      ky     0    0    0 ⎥
⎢                                                           ⎥
⎢  0      0       0      kz      0      -kx     0    0    0 ⎥
⎢                                                           ⎥
⎣  0    

In [53]:
# kx_num = 1
# ky_num = 0
# kz_num = 0.4
# n = 6

def getBerryCurvature(n, kx_num, ky_num, kz_num):
    H_num = np.matrix(H.subs({kx: kx_num, ky: ky_num, kz: kz_num}), dtype=np.complex128)
    dHdkx_num = np.matrix(dHdkx.subs({kx: kx_num, ky: ky_num, kz: kz_num}), dtype=np.complex128)
    dHdky_num = np.matrix(dHdky.subs({kx: kx_num, ky: ky_num, kz: kz_num}), dtype=np.complex128)
    evals, evecsMat = np.linalg.eigh(H_num)
    evecs = [np.matrix(evecsMat[:,i]) for i in range(len(evecsMat))]

    # Fn = 0
    # for m in range(9):
    #     if m != n:
    #         term1 = evecs[n].H @ dHdkx_num @ evecs[m]
    #         term2 = evecs[m].H @ dHdky_num @ evecs[n]
    #         term3 = evecs[m].H @ dHdkx_num @ evecs[n]
    #         term4 = evecs[n].H @ dHdky_num @ evecs[m]
    #         Fn += 1j* (term1*term2 - term3*term4)/((evals[n] - evals[m])**2)

    Fn = 0
    for m in range(9):
        if m != n:
            term1 = evecs[n].H @ dHdkx_num @ evecs[m]
            term2 = evecs[m].H @ dHdky_num @ evecs[n]
            evaldiff = evals[n] - evals[m]
            if evaldiff == 0:
                print(f"{n-4} = {evals[n]}, {m-4} = {evals[m]}")
                print(term1*term2)
                raise("Yuh")
            Fn += -2 * (term1 * term2)/((evaldiff)**2)
    Fn = Fn.imag
    
    return Fn[0,0]

In [54]:
n = 6
kz_num = 0.4
kperp = 1e-1
thetadegs = 84

theta = thetadegs/360 * 2*np.pi
kx_num = kperp * np.cos(theta)
ky_num = kperp * np.sin(theta)

print(kx_num, ky_num)

getBerryCurvature(n, kx_num, ky_num, kz_num)

0.010452846326765346 0.09945218953682733


In [1]:
def integrand(kperp):
    n = 2
    n += 4
    kz = 0.8
    return kperp * getBerryCurvature(n, kperp, 0, kz)

twopiCn, error = sp.integrate.quad(integrand, 0, 2*kc)
Cn = twopiCn / 2*np.pi
print(Cn)

NameError: name 'sp' is not defined